In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns

# Reading Data

In [ ]:
train=pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv")
train

In [ ]:
test=pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv")
test

In [ ]:
sample_submission=pd.read_csv("../input/tabular-playground-series-jan-2022/sample_submission.csv")
sample_submission

# Exploratory Data Analysis

In [ ]:
train.describe()

In [ ]:
sns.countplot(x="country",data=train)

In [ ]:
sns.countplot(x="store",data=train)

In [ ]:
sns.countplot(x="product",data=train)

In [ ]:
sns.countplot(x="product",hue="country",data=train)

In [ ]:
sns.histplot(x="country",y="num_sold",data=train)

# Data Preprocessing

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train_upd=train.drop(["row_id","date"],axis=1)
train_upd

In [ ]:
dummy_data=pd.get_dummies(train_upd,columns=["country","store","product"])
dummy_data

In [ ]:
#dropping dummies of Finland, Kaggle_Mart, Kaggle_Hat to avoid dummy variable 
Final_data=dummy_data.drop(["country_Finland","store_KaggleMart","product_Kaggle Hat"],axis=1)
Final_data

In [ ]:
X=Final_data.drop(["num_sold"],axis=1)
y=Final_data.num_sold

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.80)
#X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#rf=RandomForestClassifier(n_estimators=88,bootstrap=True)
#rf.fit(X,y)
#pred=rf.predict(X)

In [ ]:
n_estimators=[int(x) for x in np.linspace(start=10,stop=110,num=10)]
max_features=["auto","sqrt"]
max_depth=[2,6]
bootstrap=[True,False]

In [ ]:
random_grid={"n_estimators": n_estimators, 
            "max_features": max_features, 
          "max_depth": max_depth,
            "bootstrap": bootstrap }
print(random_grid)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier() 
from sklearn.model_selection import GridSearchCV
grid= GridSearchCV(estimator=model, param_grid=random_grid, cv=4,verbose=2,n_jobs=3)
grid.fit(X,y)
grid.best_params_

In [ ]:
grid.score(X,y)

In [ ]:
#pred.shape

In [ ]:
#y.shape,X.shape

In [ ]:
#from sklearn.metrics import accuracy_score
#accuracy_score(pred,y)

In [ ]:
test_data=test.drop(["row_id","date"],axis=1)
test_data

In [ ]:
test_dummy=pd.get_dummies(test_data,columns=["country","store","product"])
test_dummy

In [ ]:
# droping one dummy of each country_Finland, store_KaggleMart, product_Kaggle Hat to avoid dummy variable trap
final_test=test_dummy.drop(["country_Finland","store_KaggleMart","product_Kaggle Hat"],axis=1)
final_test

In [ ]:
prediction=grid.predict(final_test)
prediction

In [ ]:
submission=pd.DataFrame(prediction,columns=["num_sold"])
submission

In [ ]:
sample_submission=sample_submission.drop(["num_sold"],axis=1)
sample_submission

In [ ]:
submission=pd.concat([sample_submission,submission],axis=1)
submission

In [ ]:
submission.to_csv("submission.csv",index=False)

In [ ]:
!kaggle competitions submit -c tabular-playground-series-jan-2022 -f submission.csv -m "Message"